## PySpark 설치

In [1]:
!pip install pyspark==3.5.1 py4j==0.10.9.7 

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!cd /home/sangwon/.local/lib/python3.10/site-packages/pyspark/jars && wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.14/redshift-jdbc42-2.1.0.14.jar

--2024-06-22 00:22:18--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.14/redshift-jdbc42-2.1.0.14.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.212.16, 52.217.206.112, 16.182.66.8, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.212.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1048241 (1024K) [binary/octet-stream]
Saving to: ‘redshift-jdbc42-2.1.0.14.jar.6’

redshift-jdbc42-2.1 100%[===================>]   1024K   812KB/s    in 1.3s    

2024-06-22 00:22:20 (812 KB/s) - ‘redshift-jdbc42-2.1.0.14.jar.6’ saved [1048241/1048241]



In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL #1") \
    .config("spark.jars", "/home/sangwon/.local/lib/python3.10/site-packages/pyspark/jars/redshift-jdbc42-2.1.0.14.jar") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/22 00:22:26 WARN Utils: Your hostname, LAPTOP-8CK94L0D resolves to a loopback address: 127.0.1.1; using 172.21.9.191 instead (on interface eth0)
24/06/22 00:22:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/06/22 00:22:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Redshift 상의 다음 테이블을 이용하여 월별 채널별 매출과 방문자 정보 계산하기
user_session_channel, session_timestamp, session_transaction

In [4]:
# Redshift와 연결해서 DataFrame으로 로딩하기
url = "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234"

df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

df_session_transaction = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_transaction") \
    .load()

In [5]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")
df_session_timestamp.createOrReplaceTempView("session_timestamp")
df_session_transaction.createOrReplaceTempView("session_transaction")

In [6]:
df_user_session_channel.show(5)

+------+--------------------+-------+
|userid|           sessionid|channel|
+------+--------------------+-------+
|  1491|00029153d12ae1c9a...|Organic|
|    59|0002ac0d783338cfe...|  Naver|
|   117|0006246bee639c7a7...|Youtube|
|   572|0006dd05ea1e999dd...|Organic|
|   935|0007cda84fafdcf42...| Google|
+------+--------------------+-------+
only showing top 5 rows



In [7]:
df_session_timestamp.show(5)

+--------------------+--------------------+
|           sessionid|                  ts|
+--------------------+--------------------+
|0002ac0d783338cfe...|2019-07-29 12:39:...|
|00053f5e11d1fe4e4...|2019-06-24 13:04:...|
|00056c20eb5a02958...| 2019-09-26 14:50:17|
|00063cb5da1826feb...|2019-10-16 14:04:...|
|0007cda84fafdcf42...|2019-05-22 08:02:...|
+--------------------+--------------------+
only showing top 5 rows



In [8]:
df_session_transaction.show(5)

+--------------------+--------+------+
|           sessionid|refunded|amount|
+--------------------+--------+------+
|00029153d12ae1c9a...|   false|    85|
|008909bd27b680698...|   false|    13|
|0107acb41ef20db22...|   false|    16|
|018544a2c48077d2c...|   false|    39|
|020c38173caff0203...|   false|    61|
+--------------------+--------+------+
only showing top 5 rows



## 총 매출이 가장 많은 사용자 10명 찾기

In [9]:
top_rev_user_df = spark.sql("""
    SELECT userid,
        SUM(str.amount) revenue,
        SUM(CASE WHEN str.refunded = False THEN str.amount END) net_revenue
    FROM user_session_channel usc
    JOIN session_transaction str ON usc.sessionid = str.sessionid
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10""")

In [10]:
top_rev_user_df.show()

+------+-------+-----------+
|userid|revenue|net_revenue|
+------+-------+-----------+
|   989|    743|        743|
|   772|    556|        556|
|  1615|    506|        506|
|   654|    488|        488|
|  1651|    463|        463|
|   973|    438|        438|
|   262|    422|        422|
|  1099|    421|        343|
|  2682|    414|        414|
|   891|    412|        412|
+------+-------+-----------+



In [11]:
top_rev_user_df2 = spark.sql("""
SELECT
  userid,
  SUM(amount) total_amount, 
 	RANK() OVER (ORDER BY SUM(amount) DESC) rank
FROM session_transaction st
JOIN user_session_channel usc ON st.sessionid = usc.sessionid
GROUP	BY userid
ORDER BY rank
LIMIT 10""")

In [13]:
top_rev_user_df2.show()

24/06/22 00:25:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/22 00:25:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/22 00:25:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/22 00:25:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/22 00:25:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/22 00:25:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/22 0

+------+------------+----+
|userid|total_amount|rank|
+------+------------+----+
|   989|         743|   1|
|   772|         556|   2|
|  1615|         506|   3|
|   654|         488|   4|
|  1651|         463|   5|
|   973|         438|   6|
|   262|         422|   7|
|  1099|         421|   8|
|  2682|         414|   9|
|   891|         412|  10|
+------+------------+----+

